# 6 TESTING AND DEBUGGING

<strong style="color:blue">Testing</strong> is the process of running a program to try and ascertain whether or not  it <strong style="color:blue">works as intended</strong>.

<strong style="color:blue">Debugging</strong> is the process of trying to <strong style="color:blue">fix a program </strong>that you already know does not work as intended.

Good programmers **design their programs** in ways that make them <strong style="color:blue"><b style="color:red">easy</b> to test and debug</strong>. 

The key to doing this is breaking the program up into <strong style="color:blue">separate components </strong>that can be 

   
* <strong style="color:blue">1 implemented </strong>

* <strong style="color:blue">2 tested </strong>

* <strong style="color:blue">3 debugged</strong> 

**independently** of other components


## 6.1 Testing

The most important thing to say about testing is that its purpose is to  
 
 > **show that bugs exist , not to show that a program is bug-free**. 

Edsger Dijkstra,

 “Program testing can be used to show the presence of bugs, but never to show
their absence!”

Albert Einstein, 

“No amount of experimentation can ever prove me right; a single experiment can prove me wrong.”

Why is this so? Even the simplest of programs has **billions of possible inputs**. Consider, for example, a program that purports to meet the specification

```python
def isBigger(x, y):
    """Assumes x and y are ints
    Returns True if x is less than y and False otherwise."""
```
Running it on **all pairs of integers** would be, to say the least, **tedious**. 

The best we can do is to run it on **pairs of integers** that have a reasonable probability of producing the wrong answer if there is a bug in the program.

The **key** to testing is  <b style="color:blue">finding</b> a collection of inputs, called  <b style="color:blue">a test suite</b>, that has a high likelihood of  <b style="color:blue">revealing bugs, yet does not take too long to run</b>.

The key to doing this is <b style="color:blue">partitioning</b> the space of all possible inputs into **subsets** that provide equivalent
information about the correctness of the program, and then constructing **a test suite** that contains `at least one input from each partition`.

<hr style="height:2px;color:blue"/>

Consider, for example, **isBigger(x, y).** The set of possible inputs is all pairwise combinations of integers. One way to partition this set is into these **seven** subsets:
* x positive, y positive
* x negative, y negative
* x positive, y negative
* x negative, y positive
* x = 0, y = 0
* x = 0, y ≠ 0
* x ≠ 0, y = 0

If one tested the implementation on at least one value from each of these subsets, there would be reasonable probability (but no guarantee) of exposing a
bug if one exists.

<hr style="height:2px;color:blue"/>

For most programs, `finding a good partitioning of the inputs` is far easier said than done. 

Typically, people rely on **heuristics** based on exploring different paths through some combination of the `code` and the `specifications`. 

* <b>Glass-Box</b>  testing: Heuristics based on exploring paths through the `code`

* <b>Black-Box</b> testing:  Heuristics based on exploring paths through the `specification`

### 6.1.1 Black-Box Testing

In principle,black-box tests are constructed `without looking at the code` to be tested.

Black-box testing allows <b>testers</b> and <b>implementers</b> to be drawn from `separate populations`.

* <b>testers</b> -- separate ---  <b>implementers</b>

<b>Positive feature of black-box</b>

1. This independence <b>reduces</b> the likelihood of generating <b>test suites</b> that exhibit <b>mistakes</b> that are correlated with **`mistakes` in the code**. 

2. black-box testing is **robust** with respect to `implementation` **changes**.


### Good ways to generate black-box test data

1 **explore paths** through a <b>specification</b>. 

Consider, the specification

```python
def sqrt(x, epsilon):
    """Assumes x, epsilon floats
      x>= 0
      epsilon > 0
    Returns result such that
       x-epsilon <= result*result <= x+epsilon"""   
```
There seem to be only `two distinct paths` through this specification: one corresponding to `x = 0` and one corresponding to `x > 0`. 

However, common sense tells us that while it is necessary to test these two cases, **it is hardly sufficient**

#### `Boundary conditions` should also be tested.

When looking at **lists**, this often means looking at the `empty` list, a list with exactly `one element`, and a list `containing lists.`

#####  The boundary condition dealing with `numbers

When dealing with `numbers`, it typically means looking at `very small` and `very large` values as well as `“typical”` values. 

For <b style="color:blue">sqrt</b>, for example, it might make sense to try values of `x` and `epsilon` similar to those in Figure6.1

![figure61](./img/fig61.jpg)

The remaining rows test `extremely large and small values` of **x and epsilon**. 

If any of these tests **fail**, something needs to be fixed.

**Perhaps**

* there is **a bug in the code** that needs to be fixed, or

* perhaps the **specification** needs to be changed so that it is easier to meet. It might, for example, be unreasonable to expect to find an approximation of a square root when `epsilon` is **ridiculously small**.

##### Another important boundary condition to think about is  `aliasing`. 
  
Consider, for example, the code

In [2]:
def copy(L1, L2):
    """Assumes L1, L2 are lists
       Mutates L2 to be a copy of L1"""
    
    while len(L2) > 0: # remove all elements from L2
        L2.pop()       # remove last element of L2
    
    for e in L1:       # append L1's elements to initially empty L2
        L2.append(e)        

It will **work** most of the time

In [3]:
L1=[1,2,3,4]
L2=[2,3,4,5]
copy(L1, L2)
print(L2)
print(id(L1))
print(id(L2))

[1, 2, 3, 4]
1728813367112
1728813381320


In [4]:
L1=[1,2,3,4]
L2=[1,2,3,4]
copy(L1, L2)
print(L2)
print(id(L1))
print(id(L2))

[1, 2, 3, 4]
1728823897224
1728813381640


**BUT** when L1 and L2 refer to **the same list**(the same object)  **？** 

In [5]:
L1=[1,2,3,4]
L2=L1    #  L1 and L2 refer to the same list. aliasing: 别名混叠
copy(L1, L2)
print(L2)
print(id(L1))
print(id(L2))

[]
1728823897032
1728823897032


Any test suite that did not include a call of the form 
```python
copy(L, L)
```
would not reveal the bug

### 6.1.2  Glass-Box Testing

Black-box testing should never be skipped, but it is rarely sufficient.

**Without looking** at the internal structure of the **code**,it is **impossible** to know `which test cases` are likely to provide new information. Consider the trivial example:

* A **prime** number (or a prime) is a natural number greater than 1 that has no positive divisors other than 1 and itself

In [6]:
def isPrime(x):
    """Assumes x is a nonnegative int
       Returns True if x is prime; False otherwise"""
    if x <= 2:  # bug
        return False
    
    for i in range(2, x):
        if x%i == 0:
           return False
    
    return True

**Looking at the code**, we can see that because of the test <b style="color:blue">if x <= 2 </b>, the values `0, 1, and 2` are treated as special cases, and **therefore need to be tested**. 
    

In [7]:
isPrime(2)

False

But,

**Without looking at the code**, one might `not test` **isPrime(2)**, and would therefore not discover that the function call **isPrime(2)** returns `False`, erroneously indicating that 2 is not a prime

* Black-box testing should never be skipped, but it is `rarely sufficient`.

<b>Glass-box test suites</b> are usually much <b>easier to construct</b> than black-box test suites.

A glass-box test suite is <b>path-complete</b> 

   * if it exercises **every potential path** through the program. 

But this is typically `impossible` to achieve.because it depends upon `the number of times each loop` is executed and `the depth of each recursion`.For example, a recursive implementation of **factorial** follows a different path for each possible input(because the number of levels of recursion will differ).

Furthermore, even a `path-complete` test suite does `not guarantee` that `all` bugs will be exposed.


In [7]:
def iabs(x):
    """Assumes x is an int
       Returns x if x>=0 and –x otherwise"""
    if x < -1:     # bug: iabs(-1) will return -1.
        return -x
    else:
        return x

The **specification**  <b style="color:blue"> if x>=0</b> suggests that there are two possible cases: `x` either is <b style="color:blue">negative</b> or it <b style="color:blue">isn’t.</b>

This suggests that the set of inputs {2, -2} is sufficient to explore  <b>all paths(a path-complete test suite)</b> in the specification.
```
 2     isn’t negative

-2     negative
```

In [8]:
iabs(2)

2

In [9]:
iabs(-2)

2

**BUT**: The only problem is that this <b>path-complete</b> test suite  will not expose the fact that

* ```iabs(-1)``` will return ```-1```.

In [10]:
iabs(-1)

-1

Despite the limitations of **glass-box** testing, There are **a few rules of thumb** that are usually worth following:
    
* Exercise **both** branches of all **if** statements


* Make sure that each **except** clause (see Chapter 7) is executed


* For each **for** loop, have **test cases** in which
  
  * The loop is `not entered` (e.g.,if the loop is iterating over the elements of a list, make sure that it is tested on the `empty` list),
 
  * The body of the loop is executed exactly **once**, and

  * The body of the loop is executed **more than once**.


* For each **while** loop,

   * Look at the `same` kinds of cases as when dealing with **for** loops.
   
   * Include test cases corresponding to all possible ways of **exiting** the loop.
     For example, for a loop starting with
     ```python
     while len(L) > 0 and not L[i] == e
    ```
    find cases where the loop exits because `len(L) is greater than zero` and cases where it exits because `L[i] == e`.


* For **recursive functions**, include test cases that cause the function to return with `no recursive` calls, exactly `one recursive` call, and `more than one` recursive call.

### 6.1.3 Conducting Tests

**Testing** is often thought of as occurring in **two phases**: <b style="color:blue">unit testing</b> and <b style="color:blue">integration testing</b>

##### Unit testing

During this phase testers construct and run tests designed to ascertain whether **individual units** of code (e.g., functions) work properly.

One should always **start** with **unit testing**.

#####  integration testing

This is followed by  <b>integration testing</b>,which is designed to ascertain whetherthe program as **a whole** behaves as intended.

In practice, testers **cycle** through these two phases, since failures  during integration testing lead to making changes to individual units.

**Integration testing** is almost always `more` **challenging** than unit testing.

One reason for this is that **the intended behavior of an entire program** is often considerably `harder to characterize` than the intended behavior of each of its parts. For example, characterizing the intended behavior of a word processor is considerably more challenging than characterizing the behavior of a function that counts the number of characters in a document. 

**Problems of `scale`** can also make integration testing difficult.

It is not unusual for integration tests to take hours or even days to run.

Many industrial software development organizations have a **software quality assurance (SQA)** group that is separate from the group charged with implementing the software. The mission of this group is to ensure that before the software is released it is suitable for its intended purpose. 

In some organizations the `development group` is responsible for `unit testing` and the `**QA** group` for `integration testing`.

In industry, `the testing process is often highly automated`. **Testers** do not sit at terminals typing inputs and checking outputs. Instead, they use <b style="color:blue">test drivers</b> that autonomously:

* Set up the `environment` needed to invoke the program (or unit) to be tested,

* `Invoke the program` (or unit) to be tested with a predefined or automatically generated sequence of inputs,

* `Save the results` of these invocations,

* Check the `acceptability of the result`s of the tests, and

* Prepare an appropriate `report`.

During <b style="color:blue">unit testing</b>, we often need to build <b style="color:blue">stubs(存根/桩单元)</b> as well as <b style="color:blue"> drivers(驱动单元)</b>. 
    
<b style="color:blue">Drivers</b> simulate parts of the program that `use the unit` being tested, 
    
whereas <b style="color:blue">stubs</b> simulate parts of the program `used by the unit` being tested. Stubs are useful because they allow people to test units that depend upon software or sometimes even hardware that does not yet exist. This allows teams of programmers to `simultaneously` develop and test multiple parts of a system.

#### Regression testing

One attraction of automating the testing process is that it facilitates `regression testing`.

As programmers attempt to debug a program, it is all too common to install a “fix” that `breaks something that used to work`. 

Whenever `any change` is made, no matter how small, you should check that the program `still passes all of the tests` that it used to pass.

## 6.2 Debugging

There is a charming urban legend about how the process of fixing flaws in software came to be known as debugging.

Runtime bugs can be categorized along two dimensions:

**(1)Overt → covert:**

* An **overt** bug has an **obvious** manifestation
   
* A **covert** bug has **no obvious** manifestation.
   
Many bugs fall between the two extremes
   
   
**(2) Persistent → intermittent:**

* A **persistent** bug occurs **every time** the  program is run with the same inputs.
   
* An **intermittent** bug occurs only **some of the time**, even when the program is run on the same inputs and seemingly under the same conditions.

**A)** The **best kinds of bugs** to have are **overt and persistent**.
 
>Good programmers try to write their programs in such a way that programming  mistakes lead to bugs that are both **overt and persistent**. 

>This is often called **defensive programming** : the practice of writing programs that check their own operation to catch errors **as early as possible**


**B)** The next step into the pit of undesirability is bugs that are <b>overt but intermittent</b>.
   
**C)** Programs that fail in <b>covert</b> ways are often <b>highly dangerous</b>.

Bugs that are both <b>covert and intermittent</b> are almost always the <b>hardest</b> to find and fix.

### 6.2.1 Learning to Debug

#### Debugging is a learned skill.

The good news is  that it’s not hard to learn, and it is a transferable skill.

Most programmers say that 

>the most important debugging tool is the **print** statement.

**Debugging** is the process of **searching** for an explanation of that behavior. 

The key to being consistently **good at debugging** is being **systematic** in conducting that **search**.

* 1 Start by **studying the available data**: the test results and the program text


* 2 form a **hypothesis** that you believe to be consistent with all the data


* 3 design and run **a repeatable experiment** with the potential to **refute** the hypothesis


* 4 Finally, it’s important to keep **a record of what experiments**

### 6.2.2 Designing the Experiment

Think of **debugging** as **a search process**, and 

> each **experiment** as an attempt to **reduce the size of the search space****. 

**1 One way** to reduce the size of the search space is to design an experiment 

    that can be used to decide whether **a specific region of code** is responsible for a problem uncovered during integration testing.

**2 Another way** to reduce the search space is to

**reduce the amount of test data** needed to provoke a manifestation of a bug.

### Let’s look at a contrived example to see how one

A **palindrome** is a word, phrase, number, or other sequence of characters which reads the same backward or forward

```
"race car"
"rac ecar"
```

In [11]:
def isPal(x):
    """Assumes x is a list
       Returns True if the list is a palindrome; False otherwise"""
    
    temp = x
    
    temp.reverse # backward 
    
    if temp == x:
        return True
    else:
        return False

def silly(n):
    """Assumes n is an int > 0
       Gets n inputs from user
       Prints 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise"""
    
    for i in range(n):
        
        result = []   #result is never more than one element long,
        
        elem = input('Enter element: ')
        result.append(elem)
    
    if isPal(result):
        print('Yes')
    else:
        print('No')

You could try and test it on the supplied 1000 -string input. But it might be more
sensible to begin by trying it on something smaller. In fact, it would make sense
to test it on a minimal non-palindrome, e.g.,
```
ab

ba
```
palindrome> -> No

In [2]:
silly(2)  

Enter element: a
Enter element: b
Yes


The **good** news is that it **fails** even this **simple** test, so you don’t have to type in a thousand strings - **small test data** 

The **bad** news is that you have no idea why it failed.

In this case, the **code** is **small** enough that you can probably **stare** at it and find the bug (or bugs). 

However, let’s pretend that it is **too large** to do this, and start to systematically reduce the search space - **code**

### Looking at `silly`

**1** The first thing to **check** is whether **result** has the expected value, ```['a', 'b']```

1.  We check this by inserting the statement 
```python
print(result)
```
before 
```python
 if isPal(result):
``` 

In [3]:
def silly(n):
    """Assumes n is an int > 0
       Gets n inputs from user
       Prints 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise"""
    for i in range(n):
        
        result = []  
        
        elem = input('Enter element: ')
        result.append(elem)
    
    # 1. inserting the statement print result 
    print(result) 
    
    if isPal(result):
        print('Yes')
    else:
        print('No')

When the experiment is run

In [4]:
silly(2)

Enter element: a
Enter element: b
['b']
Yes


The program 
```
prints(result)  -> ['b'] not ['a','b']
```
suggest  that something has already gone wrong. 

**2 The next step is to** 

check the input loop, <b style="color:blue"> print `result` through the `loop`.</b>

In [5]:
def silly(n):
    """Assumes n is an int > 0
       Gets n inputs from user
       Prints 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise"""
    for i in range(n):
       
        result = [] 
      
        elem = input('Enter element: ')
        result.append(elem)
        
        # 2 print result roughly halfway through the loop.
        print('loop region: ',result) #  print result roughly halfway through the loop.
    
    
    # 1 inserting the statement print result 
    print(result) 
    
    if isPal(result):
        print('Yes')
    else:
        print('No')

In [6]:
silly(2)

Enter element: a
loop region:  ['a']
Enter element: b
loop region:  ['b']
['b']
Yes


    print('loop region: ',result)

    loop region:  ['a']
    loop region:  ['b']

This quickly **reveals** that 

```python
for i in range(n):
```
result is <b style="color:blue">never more than `one element` long</b>, 

suggesting that the <b style="color:blue">initialization of result 
```python
for i in range(n):
  result = [] 
```
needs to be moved **outside** the for loop</b>. 
```python
result = [] 
for i in range(n)
```    
The corrected code for `silly` is

In [7]:
def silly(n):
    """Assumes n is an int > 0
       Gets n inputs from user
       Prints 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise"""
   
    # remove Bug 1 
    result = [] # initialization of result needs to be moved outside the for loop
    
    for i in range(n):
        elem = input('Enter element: ')
        result.append(elem)
       
        # 2 print result roughly halfway through the loop
        print('loop region: ',result) 
    
    # 1 inserting the statement print result
    print(result) 
    
    if isPal(result):
        print('Yes')
    else:
        print('No')

In [8]:
silly(2)

Enter element: a
loop region:  ['a']
Enter element: b
loop region:  ['a', 'b']
['a', 'b']
Yes


Let’s try that, and see if `result` has the `correct` value after the for loop.

It does
```
Enter element: a
loop region:  ['a']
Enter element: b
loop region:  ['a', 'b']
```
but `unfortunately` the program still printsL: **Yes**.

We have reason to believe that **a second bug** lies **below the print statement**.

```python
if isPal(result):
```

lets look <b>isPal</b>

we `insert` the line 
```python
print('\n temp,x',temp,x)
```
before that line:
```python
if temp == x:
```

In [9]:
def isPal(x):
    """Assumes x is a list
       Returns True if the list is a palindrome; False otherwise"""
    temp = x
    
    temp.reverse 
    
    # 3 insert the line print temp, x before that line: if temp == x:
    print('\n temp,x',temp,x)
    
    if temp == x:
        return True
    else:
        return False
    
def silly(n):
    """Assumes n is an int > 0
       Gets n inputs from user
       Prints 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise"""
     
    result = [] # initialization of result needs to be moved outside the for loop
    
    for i in range(n):
        elem = input('Enter element: ')
        result.append(elem)
    
        print('loop region: ',result) # 2 print result roughly halfway through the loop
    
    print(result) # 1 inserting the statement print result
    
    if isPal(result):
        print('Yes')
    else:
        print('No')

In [10]:
silly(2)

Enter element: a
loop region:  ['a']
Enter element: b
loop region:  ['a', 'b']
['a', 'b']

 temp,x ['a', 'b'] ['a', 'b']
Yes


discover that both `temp` and `x` have the <b>non-reverse</b> value

```
['a', 'b'].
```

A quick inspection of the code reveals that in `isPal` 

we wrote
```python
temp.reverse
```
rather than
```python
temp.reverse()
```

the evaluation of `temp.reverse` returns the built-in `reverse method object` for lists, but does not `invoke` it.

** returns the built-in the method object**

In [14]:
r=['a','b']
r=r.reverse
print(r)

<built-in method reverse of list object at 0x7f8ab1785048>


`invoke` the method: `method()`

In [12]:
r=['a','b']
r.reverse() ## method()
print(r)

['b', 'a']


In [15]:
def isPal(x):
    """Assumes x is a list
       Returns True if the list is a palindrome; False otherwise"""
    temp = x  
    
    # remove Bug2:  
    temp.reverse() # temp.reverse -> temp.reverse()
    
    print('\n temp,x',temp,x)
    
    if temp == x:
        return True
    else:
        return False

In [16]:
silly(2)

Enter element: a
loop region:  ['a']
Enter element: b
loop region:  ['a', 'b']
['a', 'b']

 temp,x ['b', 'a'] ['b', 'a']
Yes


We run the test again, and now it seems that **both** `temp and x` have the <b>reverse</b> value 

['b', 'a']. 

We have now **narrowed** the bug to **one line**.

It seems that **temp.reverse()** `unexpectedly` changed the value of **x**

<b style="color:blue">An `aliasing` bug has bittenus</b>: 

>`temp` and `x` are names for the same list, 

both before and after the list gets reversed. 

One way to fix it is to replace the first assignment statement in `isPal` by 
```python
temp = x[:]
```
which causes a **copy** of x to be made. (Ref **P63 5.2.1 Cloning**)

In [17]:
def isPal(x):
    """Assumes x is a list
       Returns True if the list is a palindrome; False otherwise"""
   
    # remove Bug 3 An aliasing bug
    temp = x[:] #   a copy of x : temp = x ->temp = x[:]
    
    # remove Bug 2
    temp.reverse()  # temp.reverse -> temp.reverse()
    print('\n temp,x',temp,x)
    if temp == x:
        return True
    else:
        return False

In [18]:
silly(2)

Enter element: a
loop region:  ['a']
Enter element: b
loop region:  ['a', 'b']
['a', 'b']

 temp,x ['b', 'a'] ['a', 'b']
No


### THREE BUGS :

**BUG 1**: `never more than one` element long: ['b'] 

Failed to `reinitialize` a variable

```python
      for i in range(n):
        result = []
```

`initialization` of result needs to be moved `outside the for loop`

```python   
    result = [] 
    for i in range(n):
```   

**BUG 2**  both `temp` and `x` have the `non-reverse` value ['a', 'b']: 

Forgotten the `()` that turns `a reference to an object` of type function into a function `invocation`

```python
temp.reverse 
```   
add `()`

```python    
temp.reverse()
```  

**BUG 3** both `temp` and `x` have the `reverse` value [ 'b','a']: 

An `aliasing` bug: Created an unintentional alias

```python    
temp = x
```
cloning

```python
temp = x[:]
```
### Print Location: before and after:  the key locations

* **block**


* **logic**


* **operation**


Step by Step to **reduce** the size of search space 

### 6.2.3 When the Going Gets Tough

Joseph P. Kennedy, father of U.S. President John F. Kennedy, reputedly instructed his  children,
<h2 style="color:blue">When the going gets tough,the tough get going.</h2>

---
This subsection contains a few pragmatic hints about what do when the debugging gets tough.
* <b style="color:blue">Look for the usual suspects</b>
    * Passed arguments to a function in the **wrong order**

    * **Misspelled** a name, e.g., typed a lowercase letter when you should have typed an uppercase one
   
    * **Failed to reinitialize** a variable
   
    * Tested that two floating point values are **equal (==)** instead of **nearly** equal (remember that floating point arithmetic is not the same as the arithmetic you learned in school)
   
    * Tested for value equality (e.g., compared two lists by writing the expression L1 == L2) when you meant **object equality** (e.g.id(L1) == id(L2))
  
    * Forgotten that some built-in function has a **side** effect
  
    * **Forgotten the ()** that turns a reference to an object of type function into a function **invocation**
  
    *  Created an **unintentional alias* 
  
    *  Made any other mistake that is **typical for you**
---  
* <span style="color:blue">Stop asking yourself why the program isn’t doing what you want it to.Instead, ask yourself why it is doing what it is.</span>. That should be an easier question to answer, and will probably be a good first step in figuring out how to fix the program.


* <span style="color:blue">Keep in mind that the bug is probably not where you think it is</span>.If it were, you would probably have found it long ago. One practical way to go about deciding where to look is asking where the bug cannot be. As Sherlock Holmes said,“Eliminate all other factors, and the one which remains must be the truth.”


* <span style="color:blue">Try to explain the problem to somebody else</span> We all develop blind spots. It is often the case that merely attempting to explain the problem to someone will lead you to see things you have missed. A good thing to try to explain is why the bug
cannot be in certain places.


* <span style="color:blue">Don’t believe everything you read</span>. In particular, don’t believe the documentation. The code may not be doing what the comments suggest.


* <span style="color:blue">Stop debugging and start writing documentation</span> This will help you approach the
problem from a different perspective.


* <span style="color:blue">Walk away, and try again tomorrow</span> This may mean that bug is fixed later in time than if you had stuck with it, but you will probably spend a lot less of your time looking for it. That is, it is possible to trade latency for efficiency. (**Students, this is an excellent reason to start work on programming problem sets earlier rather than later!**)


### 6.2.4 And When You Have Found “The” Bug

When you think you have found a bug in your code, the temptation to start coding and testing a fix is almost irresistible. <b style="color:blue">It is often better, however, to slow down a little.</b> Remember that the goal is not to fix one bug, but to move rapidly and efficiently towards a bug-free program.

<span style="color:blue"> Ask yourself if this bug explains all the observed symptoms, or whether it is just the tip of the iceberg</span>. 

If the latter, it may be better to think about taking care of this bug in concert with other changes. Suppose, for example, that you have discovered that the bug is the result of having accidentally `mutated a list`. You could circumvent the problem locally, perhaps by making a copy of the list. Alternatively, you could consider `using a tuple instead of a list` (since tuples are immutable), perhaps eliminating imilar bugs elsewhere in the code.

<span style="color:blue">Before making any change, try and `understand the ramification` of the proposed “fix.”</span>.

Will it break something else? Does it introduce excessive complexity? Does it offer the opportunity to tidy up other parts of the code?


<span style="color:blue">Always make sure that you can get back to where you are </span>.

There is nothing more frustrating than realizing that a long series of changes have left you farther from the goal than when you started, and having no way to get back to where you started.Disk space is usually plentiful. Use it to store old versions of your program:
   
**Using Verison Control for your projects**: git and github

<span style="color:blue">Finally, if there are many `unexplained` errors</span>, you might consider whether finding and fixing bugs one at a time is even the right approach. Maybe you would be better off thinking about whether there is some better way to **organize your program or some simpler algorithm** that will be **easier** to implement correctly.
    